In [9]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [10]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [11]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show(10)

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [14]:
import pyspark.sql.functions as func
from pyspark.sql.functions import year
from pyspark.sql.functions import avg

df = df.withColumn('year', year(df['date']))

In [15]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home')


In [16]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select round(avg(price), 2) as Average_price, year 
   from home where bedrooms=4 GROUP BY year ORDER BY year""").show()

+-------------+----+
|Average_price|year|
+-------------+----+
|     300263.7|2019|
|    298353.78|2020|
|    301819.44|2021|
|    296363.88|2022|
+-------------+----+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""select round(avg(price), 2) as Average_price, date_built 
   from home where bedrooms=3 and bathrooms=3 GROUP BY date_built ORDER BY date_built""").show()

+-------------+----------+
|Average_price|date_built|
+-------------+----------+
|    292859.62|      2010|
|    291117.47|      2011|
|    293683.19|      2012|
|    295962.27|      2013|
|    290852.27|      2014|
|     288770.3|      2015|
|    290555.07|      2016|
|    292676.79|      2017|
+-------------+----------+



In [18]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""select round(avg(price), 2) as Average_price, date_built 
   from home where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living >= 2000 GROUP BY date_built ORDER BY date_built""").show()

+-------------+----------+
|Average_price|date_built|
+-------------+----------+
|    285010.22|      2010|
|    276553.81|      2011|
|    307539.97|      2012|
|    303676.79|      2013|
|    298264.72|      2014|
|    297609.97|      2015|
|     293965.1|      2016|
|    280317.58|      2017|
+-------------+----------+



In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select round(avg(price), 2) as Average_price, view 
   from home where price >= 350000 GROUP BY view ORDER BY view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_price|view|
+-------------+----+
|    403848.51|   0|
|    401044.25|   1|
|    397389.25|   2|
|     398867.6|   3|
|    399631.89|   4|
|    401471.82|   5|
|    395655.38|   6|
|    403005.77|   7|
|    398592.71|   8|
|    401393.34|   9|
|    401868.43|  10|
|    399548.12|  11|
|    401501.32|  12|
|    398917.98|  13|
|    398570.03|  14|
|     404673.3|  15|
|    399586.53|  16|
|    398474.49|  17|
|    399332.91|  18|
|    398953.17|  19|
+-------------+----+
only showing top 20 rows

--- 0.2920715808868408 seconds ---


In [20]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home")

DataFrame[]

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select round(avg(price), 2) as Average_price, view 
   from home where price >= 350000 GROUP BY view ORDER BY view""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|Average_price|view|
+-------------+----+
|    403848.51|   0|
|    401044.25|   1|
|    397389.25|   2|
|     398867.6|   3|
|    399631.89|   4|
|    401471.82|   5|
|    395655.38|   6|
|    403005.77|   7|
|    398592.71|   8|
|    401393.34|   9|
|    401868.43|  10|
|    399548.12|  11|
|    401501.32|  12|
|    398917.98|  13|
|    398570.03|  14|
|     404673.3|  15|
|    399586.53|  16|
|    398474.49|  17|
|    399332.91|  18|
|    398953.17|  19|
+-------------+----+
only showing top 20 rows

--- 0.22466397285461426 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').mode("overwrite").parquet("delayed_partitioned")

In [ ]:
# 11. Read the formatted parquet data.
df.write.partitionBy('date_built').mode("overwrite").parquet("delayed_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
pdf_p=spark.read.parquet('delayed_partitioned')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("""select round(avg(price), 2) as Average_price, view 
   from home where price >= 350000 GROUP BY view ORDER BY view""").show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home")

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home')